## Imports

In [12]:
from spark_config import init_spark
from pyspark.sql import functions as F
from datetime import date
import os
import shutil

## Start Spark Session

In [13]:
spark = init_spark("Create delta lake tables")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Variables

In [14]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define o caminho da tabela Delta Bronze
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"
delta_path_cadcliente = f"{base_bronze_path}/raw_cadcliente"
delta_path_controleativo = f"{base_bronze_path}/raw_controleativo"

# Define o caminho da tabela Delta Silver
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_controle_ativo = f"{base_silver_path}/stg_controle_ativo"

# Dimensões
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_dim_tempo = f"{base_gold_path}/dim_tempo"
delta_path_dim_cliente = f"{base_gold_path}/dim_cliente"

# Fato
delta_path_fato_cotacao = f"{base_gold_path}/fato_cotacao"
delta_path_fato_carteira = f"{base_gold_path}/fato_carteira"

## Create Schemas

In [ ]:
 # BRONZE - Camada de dados RAW
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS bronze
    LOCATION '{base_bronze_path}'
    COMMENT 'Camada Bronze - Dados brutos sem transformação. 
            Dados exatamente como chegam dos arquivos Excel.
            Todos os campos em STRING para preservar dados originais.'
    WITH DBPROPERTIES (
        'layer' = 'bronze',
        'data_quality' = 'raw',
        'created_date' = '2025-10-27',
        'sla' = 'near_realtime',
        'retention_days' = '30',
        'pii_data' = 'yes'
    )
""")

print("✅ Schema bronze criada com sucesso!")

In [ ]:
# Schema Silver
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS silver
    LOCATION '{base_silver_path}'
    COMMENT 'Camada Silver - Dados limpos e validados'
    WITH DBPROPERTIES (
        'layer' = 'silver',
        'data_quality' = 'cleaned_and_validated',
        'created_date' = '2025-10-27',
        'sla' = 'daily',
        'retention_days' = '365',
        'pii_data' = 'masked'
    )
""")

print("✅ Schema silver criada com sucesso!")

In [ ]:
# Schema Gold
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS gold
    LOCATION '{base_gold_path}'
    COMMENT 'Camada Gold - Dados agregados e métricas'
    WITH DBPROPERTIES (
        'layer' = 'gold',
        'data_quality' = 'curated',
        'created_date' = '2025-10-27',
        'sla' = 'daily',
        'retention_days' = '2555',
        'pii_data' = 'no'
    )
""")

print("✅ Schema gold criada com sucesso!")

## Create Table Bronze

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotahist):
    shutil.rmtree(delta_path_cotahist)
    print(f"✅ Diretório removido: {delta_path_cotahist}")


## Create bronze cotahist
spark.sql("""
CREATE TABLE IF NOT EXISTS bronze.raw_cotahist (
  tipo_registro STRING,
  data_pregao STRING,
  codigo_bdi STRING,
  codigo_negociacao STRING,
  tipo_mercado STRING,
  nome_resumido_empresa STRING,
  especificacao_papel STRING,
  prazo_dias_mercado STRING,
  moeda_referencia STRING,
  preco_abertura_papel STRING,
  preco_maximo_papel STRING,
  preco_minimo_papel STRING,
  preco_medio_papel STRING,
  preco_ultimo_negocio STRING,
  preco_melhor_oferta_compra STRING,
  preco_melhor_oferta_venda STRING,
  numero_negocios_efetuados STRING,
  quantidade_total_titulos STRING,
  volume_total_titulos STRING,
  preco_exercicio_opcoes STRING,
  indicador_correcao_precos STRING,
  data_vencimento_opcoes STRING,
  fator_cotacao_papel STRING,
  preco_exercicio_pontos STRING,
  codigo_papel_sistema STRING,
  numero_distribuicao_papel STRING,
  nome_arquivo STRING
)
USING DELTA
""")

print("✅ Tabela raw_cotahist criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cadcliente):
    shutil.rmtree(delta_path_cadcliente)
    print(f"✅ Diretório removido: {delta_path_cadcliente}")

# Criar bronze cadastro cliente
spark.sql("""
CREATE TABLE IF NOT EXISTS bronze.raw_cadcliente (
    cpf STRING,
    nome STRING,
    data_nascimento STRING,
    email STRING,
    telefone STRING,
    nome_arquivo STRING
)
USING DELTA
""")

print("✅ Tabela raw_cadcliente criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_controleativo):
    shutil.rmtree(delta_path_controleativo)
    print(f"✅ Diretório removido: {delta_path_controleativo}")

# Criar bronze controle ativo
spark.sql("""
CREATE TABLE IF NOT EXISTS bronze.raw_controleativo (
    data STRING,
    papel STRING,
    op STRING,
    qtd STRING,
    preco STRING,
    custo_total STRING,
    total_op STRING,
    rateio STRING,
    v_liquido STRING,
    estoque STRING,
    pmedio STRING,
    gan_per STRING,
    ir_mes STRING,
    mes_ref STRING,
    vendas_mes STRING,
    cpf STRING,
    cotista STRING,
    arquivo_origem STRING
)
USING DELTA
""")

print("✅ Tabela raw_controleativo criada com sucesso!")

## Create Table Silver

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotacao_historica):
    shutil.rmtree(delta_path_cotacao_historica)
    print(f"✅ Diretório removido: {delta_path_cotacao_historica}")

## Create table stg_cotacao_historica
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.stg_cotacao_historica (
  tp_registro STRING COMMENT 'Tipo do registro no arquivo',
  dt_pregao DATE COMMENT 'Data do pregão',
  cd_bdi STRING COMMENT 'Código BDI (identificação da ação)',
  cd_negociacao STRING COMMENT 'Código de negociação do papel',
  tp_mercado STRING COMMENT 'Tipo de mercado',
  nm_empresa STRING COMMENT 'Nome resumido da empresa',
  ds_especificacao_papel STRING COMMENT 'Especificação do papel',
  nr_prazo_dias_mercado STRING COMMENT 'Prazo em dias de mercado',
  cd_moeda_referencia STRING COMMENT 'Moeda de referência',
  vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura do papel',
  vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo do papel',
  vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo do papel',
  vl_medio DECIMAL(11,2) COMMENT 'Preço médio do papel',
  vl_ultimo_negocio DECIMAL(11,2) COMMENT 'Preço do último negócio',
  vl_melhor_oferta_compra DECIMAL(11,2) COMMENT 'Preço da melhor oferta de compra',
  vl_melhor_oferta_venda DECIMAL(11,2) COMMENT 'Preço da melhor oferta de venda',
  qt_negocios_efetuados INT COMMENT 'Número de negócios efetuados',
  qt_total_titulos INT COMMENT 'Quantidade total de títulos negociados',
  vl_total_titulos DECIMAL(16,2) COMMENT 'Volume total títulos negociados',
  vl_exercicio_opcoes DECIMAL(11,2) COMMENT 'Preço de exercício de opções',
  cd_indicador_correcao STRING COMMENT 'Indicador de correção de preços',
  dt_vencimento_opcoes DATE COMMENT 'Data de vencimento das opções',
  cd_fator_cotacao STRING COMMENT 'Fator de cotação do papel',
  vl_exercicio_pontos DECIMAL(7,6) COMMENT 'Preço de exercício em pontos',
  cd_papel_sistema STRING COMMENT 'Código do papel no sistema',
  nr_distribuicao_papel STRING COMMENT 'Número de distribuição do papel',
  nm_arquivo_origem STRING COMMENT 'Nome do arquivo origem com os dados carregados',
  ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro'
)
USING DELTA
COMMENT 'Histórico de cotações de papéis da bolsa de valores'
""")

print("✅ Tabela stg_cotacao_historica criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_controle_ativo):
    shutil.rmtree(delta_path_controle_ativo)
    print(f"✅ Diretório removido: {delta_path_controle_ativo}")

# Criar bronze controle ativo
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.stg_controle_ativo (
    dt_operacao DATE COMMENT 'Data da operação de compra ou venda do ativo',
    cd_ativo STRING COMMENT 'Código do ativo (Ações ou FII) negociado',
    cd_tipo_operacao STRING COMMENT 'Tipo da operação (ex.: COMPRA, VENDA)',
    qt_operacao INT COMMENT 'Quantidade de ativos na operação',
    vl_preco_ativo DECIMAL(11,2) COMMENT 'Preço unitário do ativo na operação',
    vl_custo_total DECIMAL(11,2) COMMENT 'Custo total da operação (preço × quantidade + custos)',
    vl_total_operacao DECIMAL(11,2) COMMENT 'Valor total da operação (pode incluir taxas, líquido da operação)',
    vl_rateio DECIMAL(11,2) COMMENT 'Rateio de custos ou despesas proporcional à operação',
    vl_liquido DECIMAL(11,2) COMMENT 'Valor líquido da operação após impostos e taxas',
    qt_estoque INT COMMENT 'Quantidade de ativos em estoque após a operação',
    vl_pmedio DECIMAL(11,2) COMMENT 'Preço médio ponderado do ativo no estoque',
    vl_ganho_perda DECIMAL(11,2) COMMENT 'Ganho ou perda da operação ou do ativo',
    ir_mes DECIMAL(11,2) COMMENT 'Imposto de renda calculado para o mês da operação',
    mes_referencia STRING COMMENT 'Mês de referência da operação (ex.: "2025-10")',
    vl_vendas_mes DECIMAL(11,2) COMMENT 'Valor total de vendas do ativo no mês de referência',
    cpf STRING COMMENT 'CPF do investidor ou cotista',
    cotista STRING COMMENT 'Nome do cotista ou titular da operação',
    arquivo_origem STRING COMMENT 'Nome do arquivo Excel de origem da carga',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro'
)
USING DELTA
COMMENT 'Tabela staging com operações de ativos dos investidores';
""")

print("✅ Tabela stg_controle_ativo criada com sucesso!")

## Create Dimension Tables - Gold

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_ativo):
    shutil.rmtree(delta_path_dim_ativo)
    print(f"✅ Diretório removido: {delta_path_dim_ativo}")

# Criar dimensão Ativo
spark.sql("""
CREATE TABLE IF NOT EXISTS gold.dim_ativo_financeiro (
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    cd_ativo STRING NOT NULL COMMENT 'Código de negociação do ativo',
    nm_empresa STRING COMMENT 'Nome da empresa',
    ds_ativo STRING COMMENT 'Descrição do ativo',
    cd_tipo_mercado STRING COMMENT 'Código do tipo de mercado',
    ds_tipo_mercado STRING COMMENT 'Descrição do tipo de mercado',
    cd_isin STRING COMMENT 'Código ISIN do ativo',
    ds_tipo_ativo STRING COMMENT 'Descrição do tipo do ativo (ação, FII, etc)',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de ativos financeiros'
""")

# Inserir registro dummy para criar a estrutura física
df_inicial = spark.createDataFrame([
    ("-1", "-1", "NÃO INFORMADO", "NÃO INFORMADO", "NÃO INFORMADO", 
     "NÃO INFORMADO", "NÃO INFORMADO", "NÃO INFORMADO")
], ["sk_ativo", "cd_ativo", "nm_empresa", "ds_ativo", "cd_tipo_mercado", 
    "ds_tipo_mercado", "cd_isin", "ds_tipo_ativo"])

df_inicial = df_inicial.withColumn("ts_insercao", F.current_timestamp()) \
                       .withColumn("ts_atualizacao", F.current_timestamp())

# Salvar usando o DataFrame (isso cria a estrutura física)
df_inicial.write.format("delta").mode("append").save(delta_path_dim_ativo)

print("✅ Tabela dim_ativo_financeiro criada com sucesso!")

✅ Tabela dim_ativo_financeiro criada com sucesso!


In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_tempo):
    shutil.rmtree(delta_path_dim_tempo)
    print(f"✅ Diretório removido: {delta_path_dim_tempo}")

# Criar dimensão Tempo
spark.sql("""
CREATE TABLE IF NOT EXISTS gold.dim_tempo (
    dt_dia DATE NOT NULL COMMENT 'Data do dia',
    nr_dia INT COMMENT 'Número do dia no mês (1-31)',
    nr_dia_ano INT COMMENT 'Número do dia no ano (1-366)',
    nr_dia_semana INT COMMENT 'Número do dia na semana (1-7)',      
    nm_dia_semana STRING COMMENT 'Nome do dia da semana (Segunda-feira, Terça-feira, etc)',
    ds_periodo_semana STRING COMMENT 'Descrição do período da semana dd-mm-yyyy a dd-mm-yyyy', 
    nr_mes INT COMMENT 'Número do mês no ano (1-12)', 
    nm_mes STRING COMMENT 'Nome do mês (Janeiro, Fevereiro, etc)',
    ds_mes_ano STRING COMMENT 'Descrição do mês e ano (ex.: Outubro 2025)',
    nr_bimestre INT COMMENT 'Número do bimestre no ano (1-6)',
    ds_bimestre STRING COMMENT 'Descrição do bimestre (ex.: 1º Bimestre)',
    nr_trimestre INT COMMENT 'Número do trimestre no ano (1-4)',
    ds_trimestre STRING COMMENT 'Descrição do trimestre (ex.: 1º Trimestre)',
    nr_quadrimestre INT COMMENT 'Número do quadrimestre no ano (1-3)',
    ds_quadrimestre STRING COMMENT 'Descrição do quadrimestre (ex.: 1º Quadrimestre)',
    nr_semeste INT COMMENT 'Número do semestre no ano (1-2)',
    ds_semestre STRING COMMENT 'Descrição do semestre (ex.: 1º Semestre)',
    nr_ano INT COMMENT 'Número do ano',
    dt_ultimo_dia_util_mes DATE COMMENT 'Data do último dia útil do mês',
    dt_ultimo_dia_mes DATE COMMENT 'Data do último dia do mês',
    fl_feriado BOOLEAN COMMENT 'Flag que indica se é feriado nacional',
    fl_fim_semana BOOLEAN COMMENT 'Flag que indica se é fim de semana',
    fl_dia_util BOOLEAN COMMENT 'Flag que indica se é dia útil',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
)
USING DELTA
COMMENT 'Dimensão de tempo'
""")

print("✅ Tabela dim_tempo criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_cliente):
    shutil.rmtree(delta_path_dim_cliente)
    print(f"✅ Diretório removido: {delta_path_dim_cliente}")

# Criar dimensão Cliente
spark.sql("""
CREATE TABLE IF NOT EXISTS gold.dim_cliente (
    sk_cliente STRING NOT NULL COMMENT 'Chave surrogate da dimensão cliente',
    cd_cpf_pessoa STRING NOT NULL COMMENT 'Código do cpf da pessoa',
    nm_cliente STRING COMMENT 'Nome do cliente',
    dt_nascimento DATE COMMENT 'Data de nascimento',
    ds_email STRING COMMENT 'Endereço de email',
    nr_telefone STRING COMMENT 'Número de telefone',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de clientes'
""")

# Inserir registro dummy para criar a estrutura física
df_inicial = spark.createDataFrame([
    ("-1", "-1", "NÃO INFORMADO", date(2100, 1, 1), "NÃO INFORMADO", "NÃO INFORMADO")
], ["sk_cliente", "cd_cpf_pessoa", "nm_cliente", "dt_nascimento", "ds_email", "nr_telefone"])

df_inicial = df_inicial.withColumn("ts_insercao", F.current_timestamp()) \
                       .withColumn("ts_atualizacao", F.current_timestamp())

# Salvar usando o DataFrame (isso cria a estrutura física)
df_inicial.write.format("delta").mode("append").save(delta_path_dim_cliente)

print("✅ Tabela dim_cliente criada com sucesso!")

✅ Tabela dim_cliente criada com sucesso!


## Create Fact Table - Gold

In [ ]:
# Remove diretório existente da tabela fato
if os.path.exists(delta_path_fato_cotacao):
    shutil.rmtree(delta_path_fato_cotacao)
    print(f"✅ Diretório removido: {delta_path_fato_cotacao}")

# Criar tabela fato de cotações
spark.sql("""
CREATE TABLE IF NOT EXISTS gold.fato_cotacao (
    dt_pregao DATE NOT NULL COMMENT 'Data do pregão',
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura',
    vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo',
    vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo',
    vl_medio DECIMAL(11,2) COMMENT 'Preço médio',
    vl_ultimo_negocio DECIMAL(11,2) COMMENT 'Preço do último negócio',
    qt_negocio INT COMMENT 'Quantidade de negócios realizados',
    qt_titulo INT COMMENT 'Quantidade de títulos negociados',
    vl_volume DECIMAL(16,2) COMMENT 'Volume financeiro negociado',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ano_pregao INT NOT NULL COMMENT 'Ano do pregão (partição)',
    mes_pregao INT NOT NULL COMMENT 'Mês do pregão (partição)'
)
USING DELTA
PARTITIONED BY (ano_pregao, mes_pregao)
COMMENT 'Fato de cotações de ativos'
""")

print("✅ Tabela fato_cotacao criada com sucesso!")

In [ ]:
# Remove diretório existente da tabela fato
if os.path.exists(delta_path_fato_carteira):
    shutil.rmtree(delta_path_fato_carteira)
    print(f"✅ Diretório removido: {delta_path_fato_carteira}")

# Criar tabela fato de carteira
spark.sql("""
CREATE TABLE IF NOT EXISTS gold.fato_carteira (
    dt_carteira DATE NOT NULL COMMENT 'Data da carteira',
    sk_cliente STRING NOT NULL COMMENT 'Chave surrogate da dimensão cliente',
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    qt_ativo INT COMMENT 'Quantidade de ativos na carteira',
    vl_ativo DECIMAL(11,2) COMMENT 'Preço do ativo',
    vl_medio DECIMAL(11,2) COMMENT 'Preço médio do ativo',
    vl_total_investido DECIMAL(16,2) COMMENT 'Valor total investido',
    vl_total_carteira DECIMAL(16,2) COMMENT 'Valor total da carteira',      
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro'
)
USING DELTA
COMMENT 'Fato de carteira de ativos'
""")

print("✅ Tabela fato_carteira criada com sucesso!")

## Stop Spark Session

In [16]:
# Encerra a SparkSession
spark.stop()